In [1]:
import pandas as pd
import numpy as np
import csv 
import random
import math

In [2]:
attributes=[]
rows=[]
# alpha=0.2
n=5

with open('heart.csv','r') as csvfile:
    csvreader=csv.reader(csvfile)
    attributes=next(csvreader)
   
    for row in csvreader:
        rows.append(row)
for i in range(len(rows)):
    for j in range(len(rows[i])):
        rows[i][j]=float(rows[i][j])
print(type(rows[1][0]))
f=len(attributes)-1
print(f)

<class 'float'>
13


In [3]:
def shuffle(data):
    for i in range(len(data)-1,0,-1):
        j=random.randint(0,i)
        data[i],data[j]=data[j],data[i]
    return data

In [4]:
def crossfold(data,i,k):
    l=len(data)
    start_test=((i-1)*l)//k
    end_test=(i*l)//k
    if start_test==0:
        start_train=end_test
        end_train=l
        return data[start_train:end_train],data[start_test:end_test]
    elif end_test==l:
        start_train=0
        end_train=start_test
        return data[start_train:end_train],data[start_test:end_test]
    else:
        s1=0
        e1=start_test
        s2=end_test
        e2=l
        x=[]
        for i in range(e1):
            x.append(data[i])
        for i in range(s2,e2):
            x.append(data[i])
        return x,data[start_test:end_test]
        

In [5]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [12]:
def train_model(data,test,alpha):
    wij=[]
    bj=[]
    for i in range(f):
        w=[]
        for j in range(n):
            w.append(1/(5*f))
        wij.append(w)
    wjk=[]
    for i in range(n):
        wjk.append(1/5)
        bj.append(1/6)
    bk=1/6
    i=[0 for i in range(n+1)]
    o=[0 for i in range(n+1)]
    err=[0 for i in range(n+1)]
    
    for iter in range(500):
        for example in data:
            if example[f]==1:
                t=1
            else:
                t=0
            for node in range(n):
                summ=0
                for att in range(f):
                    summ+=wij[att][node]*example[att]
                summ+=bj[node]
                i[node]=summ
                o[node]=sigmoid(i[node])
            summ=0
            for node in range(n):
                summ+=wjk[node]*o[node]
            summ+=bk
            i[n]=summ
            o[n]=sigmoid(i[n])
            err[n]=o[n]*(1-o[n])*(t-o[n])
            
            for node in range(n):
                err[node]=o[node]*(1-o[node])*err[n]*wjk[node]
            for att in range(f):
                for node in range(n):
                    wij[att][node]=wij[att][node]+(alpha*err[node]*example[att])
            for node in range(n):
                wjk[node]=wjk[node]+alpha*err[n]*o[node]
            bk=bk+alpha*err[n]
            for node in range(n):
                bj[node]=bj[node]+alpha*err[node]
#         print("Error is {}".format(err[n]))
                
    label=[]
    for example in test:
        i=[0 for i in range(n+1)]
        o=[0 for i in range(n+1)]
        for node in range(n):
            summ=0
            for attr in range(f):
                summ+=example[attr]*wij[attr][node]
                
            summ+=bj[node]
            i[node]=summ
            o[node]=sigmoid(i[node])
        summ=0
        for node in range(n):
            summ+=wjk[node]*o[node]
        summ+=bk
        summ_sig=sigmoid(summ)
        
        # Threshold
        if summ_sig>0:
            label.append(1)
        else:
            label.append(0)
    tn=0
    tp=0
    fn=0
    fp=0
    for i in range(len(test)):
        if test[i][f]==1 and label[i]==1:
            tp+=1
        elif test[i][f]==1 and label[i]==0:
            fn+=1
        elif test[i][f]==0 and label[i]==0:
            tn+=1
        elif test[i][f]==0 and label[i]==1:
            fp+=1
#     print(tp,fn,fp,tn)
            
            
    return tp,fn,fp,tn

In [13]:
srows=shuffle(rows)
# print(srows)
k=10
acc=[]
rec=[]
pre=[]
s=0
for i in range(1,k+1):
    train,test=crossfold(srows,i,k)
    tp,fn,fp,tn=train_model(train,test,0.4)
    
    accuracy=(tp+tn)/(tp+tn+fp+fn)*100
    s+=accuracy
    if tp==0:
        if fp==0 and fn==0:
            recall=1
            precision=1
        if fp==0 and fn!=0:
            recall=0
            precision=0
        if fp!=0 and fn==0:
            recall=0
            precision=0
    else:
        recall=tp/(tp+fn)
        precision=tp/(tp+fp)
    acc.append(accuracy)
    rec.append(recall)
    pre.append(precision)
    print("{} Fold stats->>>>>".format(i))
    
    
    print("  Accuracy for the {} fold is {}".format(i,accuracy))
    print("  Recall for the {} fold is {}".format(i,recall))
    print("  Precision for the {} fold is {}".format(i,precision))

avg_acc=s/k
    
print("Average Accuracy of the model is {}".format(avg_acc))
    
    

1 Fold stats->>>>>
  Accuracy for the 1 fold is 50.0
  Recall for the 1 fold is 1.0
  Precision for the 1 fold is 0.5
2 Fold stats->>>>>
  Accuracy for the 2 fold is 53.333333333333336
  Recall for the 2 fold is 1.0
  Precision for the 2 fold is 0.5333333333333333
3 Fold stats->>>>>
  Accuracy for the 3 fold is 56.666666666666664
  Recall for the 3 fold is 1.0
  Precision for the 3 fold is 0.5666666666666667
4 Fold stats->>>>>
  Accuracy for the 4 fold is 61.29032258064516
  Recall for the 4 fold is 1.0
  Precision for the 4 fold is 0.6129032258064516
5 Fold stats->>>>>
  Accuracy for the 5 fold is 50.0
  Recall for the 5 fold is 1.0
  Precision for the 5 fold is 0.5
6 Fold stats->>>>>
  Accuracy for the 6 fold is 56.666666666666664
  Recall for the 6 fold is 1.0
  Precision for the 6 fold is 0.5666666666666667
7 Fold stats->>>>>
  Accuracy for the 7 fold is 58.06451612903226
  Recall for the 7 fold is 1.0
  Precision for the 7 fold is 0.5806451612903226
8 Fold stats->>>>>
  Accuracy f